In [1]:
import pandas as pd
import numpy as np 

company = pd.read_csv('Company_Tweet.csv')
tweets = pd.read_csv('Tweet.csv')

In [2]:
company = company[(company['ticker_symbol']=='AAPL') | (company['ticker_symbol']=='AMZN')]

In [3]:
df = tweets[['tweet_id','post_date','body']].merge(company,on='tweet_id')

In [4]:
df['post_date'] = pd.to_datetime(df['post_date'],unit='s')
df.head()

,tweet_id,post_date,body,ticker_symbol
0,550441509175443456,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",AAPL
1,550441672312512512,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,AAPL
2,550441732014223360,2015-01-01 00:01:50,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,AMZN
3,550443807834402816,2015-01-01 00:10:05,Swing Trading: Up To 8.91% Return In 14 Days h...,AAPL
4,550443808606126081,2015-01-01 00:10:05,Swing Trading: Up To 8.91% Return In 14 Days h...,AAPL


In [31]:
import yfinance as yf
from datetime import timedelta

aapl = yf.Ticker('AAPL')
amzn = yf.Ticker('AMZN')

aapl_min = min(df[df['ticker_symbol']=='AAPL']['post_date'])
aapl_max = max(df[df['ticker_symbol']=='AAPL']['post_date'])
aapl_max = aapl_max + timedelta(days=3)

amzn_min = min(df[df['ticker_symbol']=='AMZN']['post_date'])
amzn_max = max(df[df['ticker_symbol']=='AMZN']['post_date'])
amzn_max = amzn_max + timedelta(days=3)

aapl = aapl.history(start=aapl_min.date(), end=aapl_max.date())

amzn = amzn.history(start=amzn_min.date(), end=amzn_max.date())

In [34]:
aapl.reset_index(inplace=True)

In [35]:
amzn.reset_index(inplace=True)

In [36]:
aapl = aapl[['Date','Close']]
amzn = amzn[['Date','Close']]

aapl['Next Close'] = aapl['Close'].shift(-1)
amzn['Next Close'] = amzn['Close'].shift(-1)

/tmp/ipykernel_1751/3846477317.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aapl['Next Close'] = aapl['Close'].shift(-1)


In [37]:
aapl.rename(columns={'Close':'AAPL Close','Next Close':'Next AAPL Close'},inplace=True)
amzn.rename(columns={'Close':'AMZN Close','Next Close':'Next AMZN Close'},inplace=True)

/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [38]:
stocks = aapl.merge(amzn, on='Date')
stocks

,Date,AAPL Close,Next AAPL Close,AMZN Close,Next AMZN Close
0,2015-01-02,24.714512,24.018259,308.519989,302.190002
1,2015-01-05,24.018259,24.020525,302.190002,295.290009
2,2015-01-06,24.020525,24.357344,295.290009,298.420013
3,2015-01-07,24.357344,25.293205,298.420013,300.459991
4,2015-01-08,25.293205,25.320333,300.459991,296.929993
...,...,...,...,...,...
1254,2019-12-26,71.325806,71.298729,1868.770020,1869.800049
1255,2019-12-27,71.298729,71.721893,1869.800049,1846.890015
1256,2019-12-30,71.721893,72.245934,1846.890015,1847.839966
1257,2019-12-31,72.245934,73.894341,1847.839966,1898.010010


In [39]:
df['Date'] = pd.to_datetime(df['post_date'].dt.date)

In [40]:
tweets_stocks = df.merge(stocks,on='Date')
tweets_stocks

,tweet_id,post_date,body,ticker_symbol,Date,AAPL Close,Next AAPL Close,AMZN Close,Next AMZN Close
0,550804137118801921,2015-01-02 00:01:54,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,AMZN,2015-01-02,24.714512,24.018259,308.519989,302.190002
1,550804846581125121,2015-01-02 00:04:43,Either way you're a winnah. RT @dbbrakebill: i...,AAPL,2015-01-02,24.714512,24.018259,308.519989,302.190002
2,550804975631089664,2015-01-02 00:05:14,@Weeklyoptions http://Weeklyoptionplays.com we...,AAPL,2015-01-02,24.714512,24.018259,308.519989,302.190002
3,550806213911277568,2015-01-02 00:10:09,.@BenBajarin @counternotions @GlennF Thankful ...,AAPL,2015-01-02,24.714512,24.018259,308.519989,302.190002
4,550806376092807168,2015-01-02 00:10:48,perfectly trading the S&P 500 in 2014 $FB $MU ...,AMZN,2015-01-02,24.714512,24.018259,308.519989,302.190002
...,...,...,...,...,...,...,...,...,...
1852647,1212159254884433921,2019-12-31 23:51:01,Imagine calling your broker-dealer and wanting...,AAPL,2019-12-31,72.245934,73.894341,1847.839966,1898.010010
1852648,1212159275637886976,2019-12-31 23:51:06,"$AAPL yearly~ Heck of a year.. Jan. 2, 1999~ar...",AAPL,2019-12-31,72.245934,73.894341,1847.839966,1898.010010
1852649,1212159765914079234,2019-12-31 23:53:03,That $SPY $SPX puuump in the last hour was the...,AAPL,2019-12-31,72.245934,73.894341,1847.839966,1898.010010
1852650,1212160410692046849,2019-12-31 23:55:37,I don't discriminate. I own both $aapl and $ms...,AAPL,2019-12-31,72.245934,73.894341,1847.839966,1898.010010


In [41]:
tweets_stocks = tweets_stocks[['Date','body','ticker_symbol','AAPL Close','Next AAPL Close','AMZN Close', 'Next AMZN Close']]

In [42]:
aapl_tweets_stocks = tweets_stocks[tweets_stocks['ticker_symbol']=='AAPL'][['Date','body','AAPL Close','Next AAPL Close']]
aapl_tweets_stocks.rename(columns={'AAPL Close':'Close','Next AAPL Close':'Next Close'},inplace=True)

amzn_tweets_stocks = tweets_stocks[tweets_stocks['ticker_symbol']=='AMZN'][['Date','body','AMZN Close','Next AMZN Close']]
amzn_tweets_stocks.rename(columns={'AMZN Close':'Close','Next AMZN Close':'Next Close'},inplace=True)

In [43]:
aapl_tweets_stocks

,Date,body,Close,Next Close
1,2015-01-02,Either way you're a winnah. RT @dbbrakebill: i...,24.714512,24.018259
2,2015-01-02,@Weeklyoptions http://Weeklyoptionplays.com we...,24.714512,24.018259
3,2015-01-02,.@BenBajarin @counternotions @GlennF Thankful ...,24.714512,24.018259
5,2015-01-02,Variable Moving Average [LazyBear] $AAPL http:...,24.714512,24.018259
7,2015-01-02,"@fredwilson 4 sure/with cause: ""Safety used to...",24.714512,24.018259
...,...,...,...,...
1852647,2019-12-31,Imagine calling your broker-dealer and wanting...,72.245934,73.894341
1852648,2019-12-31,"$AAPL yearly~ Heck of a year.. Jan. 2, 1999~ar...",72.245934,73.894341
1852649,2019-12-31,That $SPY $SPX puuump in the last hour was the...,72.245934,73.894341
1852650,2019-12-31,I don't discriminate. I own both $aapl and $ms...,72.245934,73.894341


In [44]:
aapl_tweets_stocks.to_csv('aapl_tweets_stocks.csv')
amzn_tweets_stocks.to_csv('amzn_tweets_stocks.csv')